In [1]:
import os
import pandapower as pp
import pandas as pd
import polars as pl
from polars import col as c
import pyomo.environ as pyo
from pipelines.dig_a_plan_model_test import DigAPlan
import logging
# from pipelines.dig_a_plan_d_model import DigAPlan
from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import pandapower_to_dig_a_plan_schema
from general_function import pl_to_dict
from copy import deepcopy
import tqdm

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"

from itertools import combinations

In [2]:
LOAD_FACTOR = 1
net = pp.from_pickle(".cache/input_data/mv_example.p")

net["load"]["p_mw"] = net["load"]["p_mw"]*LOAD_FACTOR
net["load"]["q_mvar"] = net["load"]["q_mvar"]*LOAD_FACTOR

net["line"].loc[:, "max_i_ka"] = 1e6


base_grid_data = pandapower_to_dig_a_plan_schema(net)
dig_a_plan: DigAPlan = DigAPlan(
    verbose= False, big_m = 1e4, penalty_cost = 1e1
)

dig_a_plan.add_grid_data(**base_grid_data)


dig_a_plan.test_switch_combinations()

Checking switch combinations: 100%|██████████| 6188/6188 [02:30<00:00, 41.09it/s]


In [3]:
dig_a_plan.switch_combination.drop_nulls("slave_results").sort("slave_results", descending=True).write_csv(".cache/input_data/load_facotr_1_results.csv")